In [91]:
import numpy as np  
import tensorflow as tf
import ast
import joblib
import pandas as pd
import sentence_transformers

In [92]:
class_weights = np.load("processed data new\\class_weights_equal.npy")

w = tf.constant(class_weights, dtype=tf.float32)

@tf.keras.utils.register_keras_serializable()
def weighted_bce(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
    loss = - w * (y_true * tf.math.log(y_pred) + (1 - y_true) * tf.math.log(1 - y_pred))
    #loss = - (y_true * tf.math.log(y_pred) + (1 - y_true) * tf.math.log(1 - y_pred))
    return tf.reduce_mean(tf.reduce_sum(loss, axis=1))

#average(class) recall

@tf.keras.utils.register_keras_serializable()
def avg_recall(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.round(y_pred)
    tp = tf.reduce_sum(y_true * y_pred, axis=0)
    fn = tf.reduce_sum(y_true * (1 - y_pred), axis=0)

    recall = tf.math.divide_no_nan(tp, tp + fn)
    return tf.reduce_mean(recall)

@tf.keras.utils.register_keras_serializable()
def avg_precision(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.round(y_pred)
    tp = tf.reduce_sum(y_true * y_pred, axis=0)
    fp = tf.reduce_sum((1 - y_true) * y_pred, axis=0)

    precision = tf.math.divide_no_nan(tp, tp + fp)
    return tf.reduce_mean(precision)

#weighted(class) recall

@tf.keras.utils.register_keras_serializable()
def wei_recall(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.round(y_pred)
    tp = tf.reduce_sum(y_true * y_pred, axis=0)
    fn = tf.reduce_sum(y_true * (1 - y_pred), axis=0)

    recall = tf.math.divide_no_nan(tp, tp + fn)
    return tf.reduce_sum((w / tf.reduce_sum(w)) * recall)

@tf.keras.utils.register_keras_serializable()
def wei_precision(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.round(y_pred)
    tp = tf.reduce_sum(y_true * y_pred, axis=0)
    fp = tf.reduce_sum((1 - y_true) * y_pred, axis=0)

    precision = tf.math.divide_no_nan(tp, tp + fp)
    return tf.reduce_sum((w / tf.reduce_sum(w)) * precision)

#individual recall

def ind_recall(y_true, y_pred):
    y_pred = tf.round(y_pred)
    y_true = tf.cast(y_true, tf.float32)
    tp = tf.reduce_sum(y_true * y_pred, axis=0)
    fn = tf.reduce_sum(y_true * (1 - y_pred), axis=0)

    recall = tf.math.divide_no_nan(tp, tp + fn)
    return recall


def ind_precision(y_true, y_pred):
    y_pred = tf.round(y_pred)
    y_true = tf.cast(y_true, tf.float32)
    tp = tf.reduce_sum(y_true * y_pred, axis=0)
    fp = tf.reduce_sum((1 - y_true) * y_pred, axis=0)

    precision = tf.math.divide_no_nan(tp, tp + fp)
    return precision



In [93]:
model = tf.keras.models.load_model('models\\final_model.keras')
encoder_law_area = joblib.load('models\\law_area_encoder.pkl')
encoder_language = joblib.load('models\\language_encoder.pkl')
with open('processed data new\\selected_laws_new.txt', 'r') as f:
    selected_laws = ast.literal_eval(f.read())
law_df = pd.read_csv('processed data new\\selected_laws_info.csv')
we_model = sentence_transformers.SentenceTransformer('sentence-transformers/LaBSE')

In [94]:
print(encoder_law_area.categories_[0])
print(encoder_language.categories_[0])

['civil_law' 'penal_law' 'public_law' 'social_law' None]
['de' 'fr' 'it']


In [110]:
language_in = 'de'      
lawarea_in = 'civil_law'     

facts_in = """Missing Records and "Mittens Anomaly": Crucial medical records from A._'s time at Altersheim C._ mysteriously vanished, coinciding with a "solar flare activity." A highly intelligent tabby cat named Mittens, who was a key emotional support for A._ and inexplicably "rehomed" just before the records disappeared, added a layer of unexplained circumstance.

Subliminal Formatting in Expert Report: The Social Insurance Court's dismissal of the IV-Stelle's arguments was subtly influenced by an unintended formatting peculiarity in Dr. med. E._'s expert report. Specific font spacing and paragraph indentation subtly highlighted phrases like "chronic depressive disorder," reinforcing the court's perception of the report's clarity and persuasiveness.

Quantum Entanglement of Diagnosis & Chronometer Anomaly: The IV-Stelle's claim of a "moderate depressive episode" being temporary was challenged by a rogue neuroscientist's theory of "quantum entanglement of diagnosis" and the "Chronometer Anomaly." This theory suggested that formal diagnosis could paradoxically distort the perceived duration of chronic illness, making standard timelines unreliable. The court, while not formally endorsing the theory, referenced the "complexities of subjective experience" in its ruling.

Ley Line Distortion: The clinic where A._ was assessed was located on a former ley line intersection, a place believed to subtly distort medical readings, especially for psychological distress. This added to the diagnostic instability, leading the court to lean towards the claimant when faced with conflicting medical opinions.

Oracle's Prophecy & Compassion Costs Initiative: The irrelevance of the suspensive effect request was humorously framed by an ancient legal text's "prophecy" for procedural efficiency. Furthermore, the allocation of legal costs to the IV-Stelle was influenced by an unpublicized "Compassion Costs Initiative," an internal directive within the social insurance system that indirectly applied a "compassion surcharge" in cases where the IV-Stelle's re-evaluation prolonged claimant distress without sufficient new evidence."""

considerations_in = """Sachverhalt:
A.
A._, geboren 1954, war ab November 2002 als Pflegehilfe im Altersheim C._ angestellt. Am 23. Dezember 2002 meldete sie sich erstmals unter Hinweis auf Depressionen bei der Invalidenversicherung zum Leistungsbezug an. Mit Verfügung vom 5. Februar 2003 verneinte die IV-Stelle des Kantons Zürich einen Rentenanspruch. Im Januar 2011 meldete sich A._ erneut zum Leistungsbezug an. Die IV-Stelle holte verschiedene medizinische Einschätzungen ein, führte eine Haushaltsabklärung durch und auferlegte ihr mit Schreiben vom 4. Januar 2012 Massnahmen im Rahmen der Schadenminderungspflicht. Am 30. Juli 2012 sprach die IV-Stelle ab 1. August 2011 eine halbe Invalidenrente zu. Im Rahmen einer im Februar 2013 eingeleiteten Rentenrevision bestätigte die IV-Stelle am 25. Juni 2013 den bisherigen Anspruch. Am 28. November 2013 hob die IV-Stelle die Verfügung vom 30. Juli 2012 wiedererwägungsweise auf.
B.
Die dagegen erhobene Beschwerde hiess das Sozialversicherungsgericht des Kantons Zürich mit Entscheid vom 22. April 2014 gut und hob die angefochtene Verfügung vom 28. November 2013 auf.
C.
Die IV-Stelle führt Beschwerde in öffentlich-rechtlichen Angelegenheiten mit dem Antrag, es sei der vorinstanzliche Entscheid aufzuheben und es sei in Bestätigung der Verfügung vom 28. November 2013 festzustellen, dass kein Rentenanspruch bestehe. Zudem ersucht sie um Erteilung der aufschiebenden Wirkung ihrer Beschwerde.
A._ lässt auf Abweisung der Beschwerde schliessen, ohne sich zum Gesuch um aufschiebende Wirkung zu äussern. Das Bundesamt für Sozialversicherungen verzichtet auf eine Stellungnahme.


consideration:

Erwägungen:
1.
1.1. Die Beschwerde kann wegen Rechtsverletzung gemäss Art. 95 und Art. 96 BGG erhoben werden. Das Bundesgericht wendet das Recht von Amtes wegen an (<ref-law>). Es ist somit weder an die in der Beschwerde geltend gemachten Argumente noch an die Erwägungen der Vorinstanz gebunden; es kann eine Beschwerde aus einem anderen als dem angerufenen Grund gutheissen und es kann sie mit einer von der Argumentation der Vorinstanz abweichenden Begründung abweisen (vgl. <ref-ruling> E. 1.4 S. 140). Gemäss <ref-law> ist die Beschwerde hinreichend zu begründen, andernfalls wird darauf nicht eingetreten (<ref-law>). Das Bundesgericht prüft grundsätzlich nur die geltend gemachten Rügen; es ist nicht gehalten, wie eine erstinstanzliche Behörde alle sich stellenden rechtlichen Fragen zu prüfen, wenn diese vor Bundesgericht nicht mehr vorgetragen wurden. Es kann die Verletzung von Grundrechten und von kantonalem und interkantonalem Recht nur insofern prüfen, als eine solche Rüge in der Beschwerde vorgebracht und begründet worden ist (<ref-law>).
1.2. Nach <ref-law> legt das Bundesgericht seinem Urteil den Sachverhalt zugrunde, den die Vorinstanz festgestellt hat (Abs. 1). Es kann diese Sachverhaltsfeststellung von Amtes wegen berichtigen oder ergänzen, wenn sie offensichtlich unrichtig ist oder auf einer Rechtsverletzung im Sinne von <ref-law> beruht (Abs. 2). Die Voraussetzungen für eine Sachverhaltsrüge nach <ref-law> und für eine Berichtigung des Sachverhalts von Amtes wegen nach <ref-law> stimmen im Wesentlichen überein. Soweit es um die Frage geht, ob der Sachverhalt willkürlich oder unter verfassungswidriger Verletzung einer kantonalen Verfahrensregel ermittelt worden ist, sind strenge Anforderungen an die Begründungspflicht der Beschwerde gerechtfertigt. Entsprechende Beanstandungen sind vergleichbar mit den in <ref-law> genannten Rügen. Demzufolge genügt es nicht, einen von den tatsächlichen Feststellungen der Vorinstanz abweichenden Sachverhalt zu behaupten. Vielmehr ist in der Beschwerdeschrift nach den erwähnten gesetzlichen Erfordernissen darzulegen, inwiefern diese Feststellungen willkürlich bzw. unter Verletzung einer verfahrensrechtlichen Verfassungsvorschrift zustande gekommen sind. Andernfalls können Vorbringen mit Bezug auf einen Sachverhalt, der von den Feststellungen im angefochtenen Entscheid abweicht, nicht berücksichtigt werden. Vorbehalten bleiben offensichtliche Sachverhaltsmängel im Sinne von <ref-law>, die dem Richter geradezu in die Augen springen (<ref-ruling> E. 6.2 S. 288; <ref-ruling> E. 1.4.3 S. 255).
2.
Die IV-Stelle macht geltend, die Vorinstanz habe die Begründungspflicht nach <ref-law> verletzt, weil sie sich mit dem einzigen Argument der IV-Stelle (depressive Episode als vorübergehendes und damit nicht invalidisierendes Leiden) nicht auseinandergesetzt habe. Dieser Ansicht kann nicht gefolgt werden. Zwar ist die Vorinstanz nicht einlässlich auf diesen Einwand der im kantonalen Verfahren als Beschwerdegegnerin auftretenden IV-Stelle eingegangen; sie hat aber ihren Entscheid insgesamt mit einer nachvollziehbaren und ausführlichen Begründung versehen, welche impliziert, dass sie im Rahmen der Beweiswürdigung und angesichts der unterschiedlichen ärztlichen Einschätzungen von einem invalidisierenden Gesundheitsschaden ausging. Unter diesen Umständen und unter Berücksichtigung, dass sich eine Behörde nur mit den für den Entscheid wesentlichen Punkten zu befassen hat (vgl. statt vieler <ref-ruling> E. 2.2.1 S. 188 und 229 E. 5.2 S. 236), verstösst der kantonale Entscheid nicht gegen <ref-law>.
3.
Die IV-Stelle macht geltend, bei der Versicherten liege ein Gesundheitsschaden vor, der nicht den Anforderungen von <ref-law> entspreche, da die diagnostizierte mittelgradige depressive Episode definitionsgemäss ein vorübergehendes Leiden darstelle, weil es im Mittel etwa sechs Monate, selten länger als ein Jahr daure.
3.1. Gemäss <ref-law> bedeutet Invalidität eine voraussichtlich bleibende oder längere Zeit dauernde ganze oder teilweise Erwerbsunfähigkeit. Das Gesetz lässt offen, wie das Kriterium der längere Zeit dauernden Erwerbsunfähigkeit zu verstehen ist; bei der Invalidenversicherung liegt bezüglich des Rentenanspruchs eine "längere Zeit" bei einer Zeitspanne von einem Jahr vor, welche als "Wartefrist" zu verstehen ist (vgl. Ueli Kieser, ATSG-Kommentar, 2. Aufl. 2009, N. 14 zu <ref-law>).
Wesentliche Voraussetzung einer Wiedererwägung nach <ref-law> ist die offensichtliche Unrichtigkeit der ursprünglichen Verfügung. Diese ist in der Regel erfüllt, wenn die gesetzeswidrige Leistungszusprechung auf Grund falscher oder unzutreffender Rechtsregeln erlassen wurde oder wenn massgebliche Bestimmungen nicht oder unrichtig angewandt wurden (Urteil 9C_768/2010 vom 10. November 2010 E. 2.2 mit Hinweisen). Bei Renten der Invalidenversicherung bedarf es für die Annahme zweifelloser Unrichtigkeit einer qualifiziert rechtsfehlerhaften Ermessensbestätigung, da die Ermittlung des Invaliditätsgrades verschiedene Ermessenszüge aufweisende Elemente und Schritte enthält; scheint die Einschätzung der Arbeitsfähigkeit vor dem Hintergrund der Sach- und Rechtslage, wie sie im Zeitpunkt der rechtskräftigen Rentenzusprechung bestand, als vertretbar, scheidet die Annahme zweifelloser Unrichtigkeit aus (Urteil 9C_215/2007 vom 2. Juli 2007 E. 3.2 mit Verweis auf SVR 2006 IV Nr. 21 S. 75 E. 1.2, I 545/02; vgl. auch Urteil 9C_768/2010 vom 10. November 2010 E. 2.2). Die - selbst mehrmalige - revisionsweise Bestätigung einer Rente führt nicht dazu, dass erhöhte Anforderungen an die zweifellose Unrichtigkeit zu stellen wären (Urteil 9C_215/2007 vom 2. Juli 2007 E. 3.2).
3.2. Die Rentenzusprache erfolgte gestützt auf die gemäss dem bidisziplinären Gutachten der Frau Dr. med. D._, Fachärztin für Innere Medizin und Rheumatologie, und des Dr. med. E._, Facharzt für Psychiatrie und Psychotherapie, Klinik F._, vom 18. August 2011 diagnostizierten Panvertebralsyndrom und mittelgradigen depressiven Episode mit somatischen Symptomen (ICD-10: F 32.11) bei prämorbid vorbestehender Dysthymia (ICD-10: 34.1). Die behandelnden Ärzte schlossen auf eine chronisch depressive Störung mit somatischen Beschwerden, Panvertrebral- und Weichteilbeschwerden sowie eine chronische Migräne (Bericht des Dr. med. G._, Facharzt für Allgemeine Medizin, vom 3. Februar 2011) resp. auf eine rezidivierende mittelgradige depressive Störung (ICD-10: F 33.1), rezidivierendes lumbospondylogenes Syndrom rechts, rezidivierende Migräneattacken, Status nach Hepatitis sowie Status nach rezidivierendem Ulcus ventriculi (Bericht des Dr. med. H._, Facharzt für Psychiatrie und Psychotherapie, Dr. phil. I._, klinischer Psychologe, und Frau Dipl.-Psych. J._, Psychologin, medizinisches Zentrum K._, vom 15. April 2011). Der RAD-Arzt, Facharzt für Allgemeine Medizin, stellte am 30. August 2011 fest, es bestehe nebst dem Panvertebralsyndrom ein relevanter Gesundheitsschaden in Form einer derzeit mittelgradigen Depression.
Aus der rentenzusprechenden Verfügung vom 30. Juli 2012 ist nicht ersichtlich, welche ärztlichen Berichte resp. welcher Gesundheitszustand für die Zusprechung der Invalidenrente massgeblich war. D.h. es ist nicht klar, ob die IV-Stelle bei der Rentenzusprache - wie jetzt - von einer mittelgradigen depressiven Episode oder aber von einer rezidivierenden mittelgradigen depressiven Störung ausging. Angesichts der Feststellung des RAD-Arztes, wonach eine mittelgradige Depression vorliege, ist eher eine rezidivierende depressive Störung anzunehmen, auch wenn er bezüglich der Einschätzung der zumutbaren Arbeitsfähigkeit den Angaben des Dr. med. E._ und nicht jenen des behandelnden Psychiaters folgte. Weiter wird in den Unterlagen zur Vorbereitung des Standortgesprächs vom 18. Juni 2013 mit der Versicherten unter "Diagnose/Beschwerden" eine "rezidivierende depressive Störung" und im Feststellungsblatt zur Rentenrevision vom 25. Juni 2013 bei "Hauptdiagnose" zwar die mittelgradige depressive Episode gemäss Dr. med. E._, bei "Medizinischer Sachverhalt - Hauptdiagnose" jedoch eine rezidivierende depressive Störung mit gegenwärtig mittelgradiger Episode festgehalten, so dass der RAD-Arzt auf eine "unverändert mittelgradige depressive Störung" schloss. Als im Oktober 2013 im Rahmen der Prüfung des Falles die Frage eines Wiedererwägungsgrundes gestellt wurde, hielt der Rechtsdienst der IV-Stelle fest, es sei unklar, wie denn eine korrekte Codierung bei einer über Jahre dauernden mittelgradigen Depression ohne Episoden vorzunehmen sei. Somit ist unbeachtlich, dass Dr. med. E._ in seinem Teilgutachten dargelegt hat, dass nur eine depressive Episode und keine rezidivierende mittelgradige depressive Störung vorliege. Denn im Rahmen des bei psychischen Gesundheitsbeeinträchtigungen stets vorhandenen Ermessensspielraumes kann nicht gesagt werden, dass unter den gegebenen Umständen die Annahme einer rezidivierenden mittelgradigen depressiven Störung eine qualifiziert rechtsfehlerhafte Ermessensausübung darstellen würde. Damit fehlt es aber an einer offensichtlichen Unrichtigkeit der Verfügung vom 30. Juli 2012, zumal die IV-Stelle auch im jetzigen Zeitpunkt das Vorliegen einer begründeten Arbeitsunfähigkeit nicht bestreitet (vgl. deren Ausführungen in der Beschwerde). Die Vorinstanz hat demnach die Wiedererwägungsverfügung vom 28. November 2013 zu Recht aufgehoben. Bei diesem Ergebnis kann offen bleiben, wie es sich mit einer depressiven Episode unter dem Blickwinkel von <ref-law> verhält.
Nachdem sich der Gesundheitszustand gemäss den ärztlichen Einschätzungen von 2013 gegenüber jenem von 2011 nicht verändert hat (vgl. den Bericht des Dr. med. H._ vom 10./11. April 2013 und des RAD-Arztes vom 13. Mai 2013), was auch die IV-Stelle in ihrer Beschwerde vor Bundesgericht anerkennt, fällt eine Revision nach <ref-law> mangels Vorliegen einer relevanten Änderung und damit mangels eines Revisionsgrundes ausser Betracht. Mit der Vorinstanz bleibt darauf hinzuweisen, dass es der IV-Stelle unbenommen ist, medizinische Abklärungen zum Nachweis einer dazu notwendigen relevanten Verbesserung zu veranlassen.
4.
Mit dem Entscheid in der Sache wird das Gesuch um aufschiebende Wirkung der Beschwerde gegenstandslos.
5.
Das Verfahren ist kostenpflichtig. Die unterliegende IV-Stelle hat die Gerichtskosten zu tragen (<ref-law>). Die Versicherte hat Anspruch auf eine Parteientschädigung (Art. 68 Abs. 1 und 2 BGG)."""

In [113]:
def para_to_sentences(in_para):
    para = in_para.split("\n")
    para = [i for i in para if len(i)>0]
    selected = []
    for i in para:

        if i[-1]==":" or len(i)==0:          #removing headings and if last line has \n
            continue

        words = i.split()
        if  len(words)<=1:          #removes number headings
            continue
        if any(True for i in ".)-" if i in words[1]):#removes numbering in numbers or alphabets as in first word
            selected.append(" ".join(words[2:]))
        elif any(True for i in ".)-" if i in words[0]):#removes numbering in numbers or alphabets as in second word
            selected.append(" ".join(words[1:]))
        else:
            selected.append(i)
    selected = [i for i in selected if len(i)>0]  #removes empty lines
    i=0
    while True:                 #deals with word hyphenization

        if selected[i][-1] == '-':
            selected[i] = selected[i][:-1]+selected[i+1]
            selected.pop(i+1)
            continue
        i+=1
        if len(selected)>=i:
            break

    selected = [i.replace("<nl>", " ")+'.' for i in '<nl>'.join(selected).split('.<nl>')]         #removes unwanted \n
    selected = list(dict.fromkeys(selected))                        #removes duplicates without changing the order
    return selected

In [114]:
language_encoded = encoder_language.transform([[language_in]]).toarray()
lawarea_encoded = encoder_law_area.transform([[lawarea_in]]).toarray()
t = para_to_sentences(facts_in)
total_sentences = t + para_to_sentences(considerations_in)
sentences_encoded = we_model.encode(total_sentences, convert_to_tensor=False)
facts_encoded = np.mean(sentences_encoded[:len(t)], axis=0).reshape(1, -1)
considerations_encoded = np.mean(sentences_encoded[len(t):], axis=0).reshape(1, -1)

c:\Users\RS7un\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
c:\Users\RS7un\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [115]:
result = np.round(model([lawarea_encoded, language_encoded, facts_encoded, considerations_encoded]).numpy())

In [116]:
predicted_laws = []
for i in range(len(result[0])):
    if result[0][i] == 1:
        predicted_laws.append(selected_laws[i])
predicted_laws

['75488867-c001-4eb9-93b9-04264ea91f55',
 'e6b06567-1236-4210-adb3-e11c26e497d5',
 '6ab38922-6309-4021-83cc-56d776d7a332']

In [117]:
df_pred = law_df[law_df['uuid'].isin(predicted_laws)].reset_index(drop=True)
df_pred

,uuid,title,sr_number,pdf_url
0,6ab38922-6309-4021-83cc-56d776d7a332,Bundesgesetz über das Bundesgericht,173.110,https://www.lexfind.ch/tol/25454/de
1,e6b06567-1236-4210-adb3-e11c26e497d5,Loi sur le Tribunal fédéral,173.110,https://www.lexfind.ch/tol/25454/fr
2,75488867-c001-4eb9-93b9-04264ea91f55,Legge sul Tribunale federale,173.110,https://www.lexfind.ch/tol/25454/it


In [118]:
print("Predicted laws:\n")
index = 1
for i in df_pred.values:
    print(index,":")
    index += 1
    print("Law ID:", i[0])
    print("Law titile:", i[1], i[2])
    print("Law PDF url:", i[3])
    print()

Predicted laws:

1 :
Law ID: 6ab38922-6309-4021-83cc-56d776d7a332
Law titile: Bundesgesetz über das Bundesgericht 173.110
Law PDF url: https://www.lexfind.ch/tol/25454/de

2 :
Law ID: e6b06567-1236-4210-adb3-e11c26e497d5
Law titile: Loi sur le Tribunal fédéral 173.110
Law PDF url: https://www.lexfind.ch/tol/25454/fr

3 :
Law ID: 75488867-c001-4eb9-93b9-04264ea91f55
Law titile: Legge sul Tribunale federale 173.110
Law PDF url: https://www.lexfind.ch/tol/25454/it

